 <h1 align="center">Data Preparation</h1> 
 <h4 align="center">Jiarui Xu - jxu57@illinois.edu</h4> 

In [21]:
import pyprind
import sys
import pickle

In [22]:
data_folder = "/Volumes/backup/ccg_tweet_wikifier_data/"

## 1. Acquire Tweets

### a. Set Twitter API

In [5]:
# Set API keys
consumer_key = 'FTglCRcahFJxH3U0TVoakD3BS'
consumer_secret = 'BoNfyAic70hWkmH6jIzr4xoE48iFyIRnqvwE4NjSwpfJVwD7N2'
access_token = '3081709104-XeNjGf7h9l6G1ERuPE1l9KOX85XJcWMBflWd1P8'
access_token_secret = '9Z1czrN0jU4hs08KtxE7IeIcjNWAd1LbWqblvyvzJoRfN'

In [7]:
# Set up API
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

### b. Load File

label-trainDev.tsv -- train split of NEEL [2] dataset    
label-train.tsv -- train split of label-trainDev.tsv used in this paper (used for parameter tuning)    
label-dev.tsv -- dev split of label-trainDev.tsv used in this paper (used for parameter tuning)    
label-test.tsv -- test split of NEEL [2] dataset    

In [9]:
file_list = ["label-trainDev.tsv", "label-train.tsv", "label-dev.tsv", "label-test.tsv"]

Get a set of unique tweet ids

In [12]:
tweet_ids = set()
for file_name in file_list:
    with open ("./data/v1-NEELOnly/" + file_name, "r") as f:
        for line in f:
            segs = line.split("\t")
            tweet_id = segs[0]
            tweet_ids.add(tweet_id)

### c. Retrieve Tweets using API

Define a function to retrieve tweets

In [16]:
def fill_tweet_set(id_list):
    # set up progress bar
    bar = pyprind.ProgBar(len(id_list))
    
    tweet_set = {}
    failed = {}
    
    for id in id_list:
        bar.update()
        
        try:
            tweet_set[id] = api.get_status(id,)._json
        except:
            failed[id] = sys.exc_info()[0]
    
    return (tweet_set, failed)

In [19]:
result = fill_tweet_set(tweet_ids)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 02:01:23


save the tweet set to file

In [46]:
tweet_corpus = {}

for tweet_id in result[0].keys():
    tweet_corpus[tweet_id] = {}
    tweet_corpus[tweet_id]['tweet_info'] = result[0][tweet_id]

with open(data_folder+"Tweet/NEEL_tweets(raw).pickle", "wb") as f:
    pickle.dump(tweet_corpus, f)

A sample tweet object in json

In [47]:
tweet_corpus['93314579924393984']

{'tweet_info': {u'contributors': None,
  u'coordinates': None,
  u'created_at': u'Tue Jul 19 13:41:44 +0000 2011',
  u'entities': {u'hashtags': [],
   u'symbols': [],
   u'urls': [],
   u'user_mentions': []},
  u'favorite_count': 2,
  u'favorited': False,
  u'geo': None,
  'goldens': [{'end_idx': '10',
    'freebase_title': '/m/02_nkp',
    'mention': 'Lamar Odom',
    'start_idx': '0',
    'tid': '93314579924393984',
    'wiki_title': 'Lamar_Odom'}],
  u'id': 93314579924393984,
  u'id_str': u'93314579924393984',
  u'in_reply_to_screen_name': None,
  u'in_reply_to_status_id': None,
  u'in_reply_to_status_id_str': None,
  u'in_reply_to_user_id': None,
  u'in_reply_to_user_id_str': None,
  u'is_quote_status': False,
  u'lang': u'en',
  u'place': None,
  u'retweet_count': 33,
  u'retweeted': False,
  u'source': u'<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
  u'text': u"Lamar Odom's Car Accident Claims Life Of A 15 Year Old Boy http://on.vh1.com/pJ

# 2. Load Golden Standard

In [48]:
def golden_generator(tid, s_idx, e_idx, wiki_title, freebase, mention):
    ret = {}
    ret['tid'] = tid
    ret['start_idx'] = s_idx
    ret['end_idx'] = e_idx
    ret['wiki_title'] = wiki_title
    ret['freebase_title'] = freebase
    ret['mention'] = mention
    return ret

In [49]:
def golden_standard(file_list, tweet_corpus):
    unique_lines= set()
    
    for file_name in file_list: 
        with open ("./data/v1-NEELOnly/" + file_name, "r") as f:
            for line in f:
                unique_lines.add(line.strip())
    
    for tw in tweet_corpus.values():
        tw['goldens'] = []
        
    for line in unique_lines:
        segments = line.strip().split("\t")
        tid = segments[0]
        if tid in tweet_corpus:
            golden = golden_generator(segments[0], segments[1], segments[2],\
                                        segments[3], segments[4], segments[5])
            tweet_corpus[tid]['goldens'].append(golden)

Now loading ...

In [50]:
golden_standard(file_list, tweet_corpus)

This is an example of golden standard

In [53]:
tweet_corpus['93314579924393984']['goldens']

[{'end_idx': '10',
  'freebase_title': '/m/02_nkp',
  'mention': 'Lamar Odom',
  'start_idx': '0',
  'tid': '93314579924393984',
  'wiki_title': 'Lamar_Odom'}]

# 3. Extracting Twitter's Special Information

https://support.twitter.com/articles/166337

### a. Hashtag

In [54]:
for tw in tweet_corpus.values():
    if len(tw['tweet_info']['entities']['hashtags'])>0:
        print tw['tweet_info']['entities']['hashtags']

[{u'indices': [14, 26], u'text': u'TheBachelor'}]
[{u'indices': [113, 126], u'text': u'pastranathon'}]
[{u'indices': [69, 76], u'text': u'Prison'}, {u'indices': [124, 134], u'text': u'detention'}]
[{u'indices': [29, 40], u'text': u'compromise'}]
[{u'indices': [71, 84], u'text': u'SLAMnewswire'}, {u'indices': [85, 89], u'text': u'NBA'}]
[{u'indices': [23, 30], u'text': u'Tahrir'}, {u'indices': [95, 101], u'text': u'Egypt'}]
[{u'indices': [10, 18], u'text': u'Mubarak'}, {u'indices': [115, 121], u'text': u'Egypt'}, {u'indices': [122, 128], u'text': u'Jan25'}]
[{u'indices': [0, 6], u'text': u'Libya'}]
[{u'indices': [73, 80], u'text': u'Ravens'}]
[{u'indices': [128, 140], u'text': u'londonriots'}]
[{u'indices': [89, 94], u'text': u'Oslo'}]
[{u'indices': [0, 24], u'text': u'ThoughtsOnChatWithFelol'}, {u'indices': [132, 138], u'text': u'Egypt'}]
[{u'indices': [119, 131], u'text': u'Londonriots'}]
[{u'indices': [69, 83], u'text': u'sharonshannon'}]
[{u'indices': [101, 105], u'text': u'huh'}]
[

In [ ]:
def make_hashtag_mapping(tw_id):
    ret = {}
    for tag in tw_corpus[tw_id]['tweet_info']['entities']['hashtags']:
        ret[tag['text']] = tag
    return ret

### b. Mention

### c. Cashtag

### d. URL